# Elastic Stackの構築：状態の診断
----
稼働しているElasticsearchに異常がないか調べ、 異常がある場合の対処手順について説明します。

## 設定情報の引き継ぎ

本章のコマンドを実行するための設定を行います。  

In [1]:
%run rc.py
(USER, KEYPATH)

In [2]:
%env ES_HOST={es_host}:9200

診断の対象としたいインデックスを指定してください。

In [3]:
%env INDEX=meteorological-data-*

※この後のコマンドは、[01_03_Set_Inventory.ipynb](01_03_Set_Inventory.ipynb) で生成したインベントリに対して発行されるように構築されています。

## 診断
現在の状態を把握するために、各種の状態を診断します。  
本章の一連のコマンドを実行してください。  

「■確認内容」で異常があると診断できた場合、次章の[症状と対処](#症状と対処)の内容に従い状態を正常化してください。

### サーバマシンが稼働しているか？
サーバごとにpingコマンドを実行し、サーバが稼働しているか確認します。

In [4]:
!ansible all -m ping -i ./hosts -u $USER --private-key=$KEYPATH

**■確認内容**

サーバごとに次のような応答があれば正常です。

### プロセスが稼働しているか？
サーバごとにpsコマンドを実行し、Elasticsearchプロセスが稼働しているか確認します。

In [5]:
!ansible all -m shell -a "ps -aef |grep Elasticsearch |grep -v grep" -i ./hosts -u $USER --private-key=$KEYPATH

**■確認内容**

Elastisearchが稼働しているはずのサーバごとに、次のコマンドのプロセスが存在していれば正常です。

### ディスクの使用量に余裕があるか？
サーバごとにdfコマンドを実行し、ディスクサイズに余裕があるか確認します。

In [6]:
!ansible all -m shell -a "df" -i ./hosts -u $USER --private-key=$KEYPATH

**■確認内容**

Elasticsearchのデフォルトの設定では、ディスク使用率が85%を越えると、そのノードに新たなシャードを割り当てません。そのため、**"Use%"**の列に出力されるディスク使用率が85%を越えないようにしてください。  
ディスク使用率に余裕があれば問題ありません。

### クラスタと通信できるか？
APIを発行し、Elasticsearchのクラウドと通信可能か確認します。

In [7]:
!curl -XGET "http://$ES_HOST/"

**■確認内容**

JSON形式の応答が表示されれば問題ありません。  
表示されない場合は、クラスタの状態か、クラスタまでの通信経路に問題があります。

### クラスタに全Nodeが参加しているか？
クラスタのヘルスチェック用のAPIを発行し、参加しているノードの状態等を確認します。

In [8]:
!curl -XGET "http://$ES_HOST/_cluster/health?pretty"

**■確認内容**

出力結果から次の内容が確認できれば正常です。

1. "status" が "green" かどうか？
  - **green**  
      ：正常です。
  - **yellow**  
    ：Primary Shardは割り当てられていますが、Replica Shardが割り当てられていません。
  - **red**  
    ：特定のShardがクラスタに割り当てられていません
2. "number_of_nodes" の数が、起動しているはずの全Node数と一致しているか？
3. "number_of_data_nodes"の数が、起動しているはずのData Node数と一致しているか？

### エラーログが無いか？
各ElasticsearchプロセスがERRORメッセージをログに出力していないか確認します。

ログはデフォルトでは次の場所に出力されています。

/var/log/elasticsearch/**{クラスタ名}**.log  


その他のログも含め、ログディレクトリ配下のファイルについてERRORが出力がされていないかgrepします。

In [9]:
!ansible all -m shell -a 'grep ERROR /var/log/elasticsearch/*.log || exit 0' -i ./hosts -u $USER --private-key=$KEYPATH

**■確認内容**

    XXX.XXX.XXX.111 | FAILED | rc=1 >>

のように出力される場合は、grepした結果でERRORが0行ということなので、問題がありません。  
1行以上、何かの出力がある場合は問題があります。

### Slowログが無いか？  
各ElasticsearchプロセスがSlowログを出力していないか確認します。

Slowログはデフォルトでは次の場所に出力されています。

/var/log/elasticsearch/**{クラスタ名}**_index_indexing_slowlog.log  
/var/log/elasticsearch/**{クラスタ名}**_index_search_slowlog.log  

In [10]:
!ansible all -m shell -a 'cat /var/log/elasticsearch/*slowlog.log' -i ./hosts -u $USER --private-key=$KEYPATH

**■確認内容**

    XXX.XXX.XXX.111 | SUCCESS | rc=0 >>

のように出力される場合は、Slowログが0行ということなので、問題がありません。  
1行以上、何かの出力がある場合は問題があります。

### クラスタ内のサーバのCPU・メモリが枯渇してないか？
CPU使用率、メモリ使用率が利用できる限界に近づいていないか確認します。

In [11]:
!ansible all -m shell -a 'top n 1 b ' -i ./hosts -u $USER --private-key=$KEYPATH

**■確認内容**

CPU使用率、メモリ使用率が継続的に100%近い数値になっていないか確認します。  
継続的に100%近い数値になっている場合、リソースが枯渇していると考えられます。

### ヒープが枯渇していないか？
ノード状態を調査するAPIを発行し、ヒープの利用状況を確認します。

Nodes Stats APIを、JVMの状態を見るオプションを付与して発行します。  
APIの詳細はElasticsearch Referenceの[Nodes Stats](https://www.elastic.co/guide/en/elasticsearch/reference/current/cluster-nodes-stats.html)を参照してください。

In [12]:
!curl -XGET "http://$ES_HOST/_nodes/stats/jvm?pretty"

**■確認内容**

応答の"nodes"部分にNodeごとの状態が出力されます。  
その下の"jvm"部分にJVMの状態が出力されます。次を確認してください。

1. mem/heap_max_in_bytes に表示される最大ヒープサイズ（単位はbyte）がインストール時に指定した通りか？
2. mem/heap_used_percent に表示されるヒープ使用率（単位はパーセント）が100近くになってしまっていないか？
3. gc/collectors/old/collection_count に表示されるGCカウントが、稼働時間に比して大きすぎないか？

### 実行中の重いタスクが無いか？
タスク管理用のAPIを発行し、実行中の重いタスクが無いか確認します。

Task Management APIでクラスタ内のノードで実行中のタスクを確認することができます。  
APIの詳細はElasticsearch Referenceの[Task Management API](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/tasks.html)を参照してください。


In [13]:
!curl -XGET "http://$ES_HOST/_tasks?pretty"

**■確認内容**

応答の"nodes"部分にNodeごとの状態が出力されます。  
その下の"tasks"部分に走行しているタスクの種類と状態が出力されます。次を確認してください。

1. tasksに現れるタスクに想定外の物が無いか？
2. 特にrunning_time_in_nanos（実行時間）の長いものが無いか？
3. 大量のタスクが滞留していないか？

### インデックスが作成されているか？
日付等のインデックスで、投入したドキュメントが適切に分割・管理されているか確認します。

In [14]:
!curl -XGET http://$ES_HOST/_cat/indices?v

**■確認内容**

出力結果から次の内容が確認できれば正常です。

1. indexの列に表示されるインデックス名が想定どおりか？  
例えば日付インデックスであれば、日付形式の文字列が含まれるか？
2. 各インデックスの"health" 列が "green" かどうか？
  - **green**  
      ：正常です。
  - **yellow**  
    ：Primary Shardは割り当てられていますが、Replica Shardが割り当てられていません。
  - **red**  
    ：特定のShardがクラスタに割り当てられていません

### Shard数が設定通りか？
インデックスごとに設定されるShard数が設定した（はずの）値になっているか確認します。

In [15]:
!curl -XGET http://$ES_HOST/$INDEX/_settings?pretty

インデックスごとに次のような内容が出力されます。

**■確認内容**

"number_of_shards"の数について次を確認してください。

- 最初にセットアップしたShard数と一致しているか？
- Shard数を設定したことが無いのであれば、現状のData Node数と比べて小さくないか？

## 症状と対処
診断の章で異常があった場合、本章の内容に従い対処します。

**先頭から順に**症状を確認してください。  
症状に対処したら、いったん[診断](#診断)の章に戻り、コマンドを**全て再実行し直してください。**

これは、本章の各節にある対処が、その前までの節に症状が出ていないことを前提に記述しているためです。  
※例えば「クラスタ状態が"green"でない」節の対処は、その前にある「サーバマシンが稼働していない」節では問題が無いことを前提に記述しています。よってサーバが起動しているかの確認はしていません。

### サーバマシンが稼働していない
**■項目**：[サーバマシンが稼働しているか？](#サーバマシンが稼働しているか？)  
**■症状**：pingの応答がないマシンが存在する

**■原因**：  
ハードウェアの障害や誤操作でサーバが停止している。  
または、サーバは稼働しているが、通信経路に問題がありpingが到達しない。

**■対処**：  
- マシンが停止している場合は稼働させてください。  
なんらかのハードウェア障害が発生している場合は、それを解決してください。
- マシンが停止していない場合は、通信経路の状態を確認し、問題を是正してください。

### プロセスが稼働していない
**■項目**：[プロセスが稼働しているか？](#プロセスが稼働しているか？)  
**■症状**：稼働していないElasticsearchプロセスが存在する

**■原因**：  
なんらかのエラー発生、または誤操作によりプロセスがダウンしている。もしくは、そもそも起動していない。

**■対処**：  
- 「[エラーログが無いか？](#エラーログが無いか？)」でエラーが発生していない場合は、クラスタを起動してください。  
次のコマンドでクラスタを起動できます。

In [16]:
#!ansible-playbook playbooks/start_cluster.yml -i ./hosts -u $USER --private-key=$KEYPATH

- エラーログが出力されている場合は「[エラーが出力されている](#エラーが出力されている)」の対処に進んでください。

### ディスクの使用量に余裕がない
**■項目**：[ディスクの使用量に余裕があるか？](#ディスクの使用量に余裕があるか？)  
**■症状**：ディスクの残量が少ない

**■原因**：  
Elasticsearchのプロセスか、それ以外のプロセスによる使用量の増加。

**■対処**：  
どのプロセスがディスクサイズを占有しているのか確認します。

In [17]:
!ansible all -m shell -a "sudo du -m --max-depth=1 / || exit 0" -i ./hosts -u $USER --private-key=$KEYPATH
# MB単位で、ルートから1階層のみ計測しています。必要に応じてオプションを変更してください。

- Elasticsearchのデフォルトの設定では、ディスク使用率が85%を越えると、そのノードに新たなシャードを割り当てません。そのため、ディスク使用率が85%を越えないようにしてください。
- Elasticsearch以外が占有している場合は、そちらのデータ削減を検討してください。
- Elasticsearchが占有している場合は、不要になったインデックス等の削除を検討してください。
- 削除できない場合は、Data Nodeを追加してください。  
[01_50_Add_Data_Node.ipynb](01_50_Add_Data_Node.ipynb)を実施してください。


- なおElasticsearchには、ディスク使用率が100%にならないようwater markを設け、状態に応じてShardを割り振る機能があります。
  - 詳細はElasticsearch Referenceの[Disk-based Shard Allocation](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/disk-allocator.html)を参照してください。
  - 例えばディスク使用率が80%を超えた場合、そのNodeには新しいShardを割り当てなくし、  
ディスクの空き領域が50GBあった場合、Shardを再配置するelasticsearch.ymlの設定は次の通りです。


### クラスタと通信できない
**■項目**：[クラスタと通信できるか？](#クラスタと通信できるか？)  
**■症状**：APIの応答がない、または異常な内容

**■原因**：  
稼働しているElasticsearchに外部通信が許可されていない。

**■対処**：  
- セキュリティの設定等で、クラスタと外部との通信が阻害されていないか確認してください。  
Elasticsearchは外部との通信用に**9200**番ポートを利用します。  
9200番ポートが許可されているか確認してください。
  - 例えば、AWSの場合はセキュリティグループの設定を確認してください。  
  次のコマンドで "FromPort"と"ToPort"の値域に**9200**を含むブロックを確認してください。  
  そのブロックの"IpRanges"配下に通信先のサーバが登録されていなければ、外部からの通信が遮断されます。

In [18]:
#!aws ec2 describe-security-groups

#VPCのIDがわかっている場合は次のコマンドで表示を絞り込めます。
#VPC_ID='vpc-bd3b81d9'
#!aws ec2 describe-security-groups --filters "Name=vpc-id,Values=$VPC_ID"

### クラスタにNodeが参加していない
**■項目**：[クラスタに全Nodeが参加しているか？](#クラスタに全Nodeが参加しているか？)  
**■症状**："status"がgreenでない、または "number_of_data_nodes"の数が、起動しているはずのData Node数より少ない

**■原因**：  
クラスタ内部の通信が阻害されており、検出できないNodeがある。  
※ここまでの対処で、サーバもプロセスも稼働しており、pingも正常に応答している前提

**■対処**：  
- セキュリティの設定等で、クラスタ内部の通信が阻害されていないか確認してください。  
Elasticsearchはクラスタ内部の通信用に**9300**番ポートを利用します。9300番ポートが許可されているか確認してください。
  - 例えばAWSの場合はセキュリティグループの設定を確認してください。  
  次のコマンドで "FromPort"と"ToPort"の値域に**9300**を含むブロックを確認してください。  
  そのブロックの"IpRanges"配下に通信先のサーバが登録されていなければ、内部の通信が遮断されます。

In [19]:
#!aws ec2 describe-security-groups

#VPCのIDがわかっている場合は次のコマンドで表示を絞り込めます。
#VPC_ID='vpc-bd3b81d9'
#!aws ec2 describe-security-groups --filters "Name=vpc-id,Values=$VPC_ID"

### エラーログが出力されている（ディスクフル）
**■項目**：[エラーログが無いか？](#エラーログが無いか？)  
**■症状**：ログに次のようなディスクフルを示すメッセージが出力されている

**■原因**：  
Elasticsearchのプロセスか、それ以外のプロセスによる使用量の増加。

**■対処**：  
適切に「[ディスクの使用量に余裕がない](#ディスクの使用量に余裕がない)」の節での対処を実施している場合は、おそらくこの問題は既に解決しています。  
「[ディスクの使用量に余裕がない](#ディスクの使用量に余裕がない)」の対処後、同様のログが出力されるか確認してください。

### エラーログが出力されている（OSのメモリマップ設定誤り）
**■項目**：[エラーログが無いか？](#エラーログが無いか？)  
**■症状**：ログに次のようなOSのメモリマップ設定誤りを示すメッセージが出力されている

**■原因**：  
カーネルパラメータvm.max_map_countの設定が誤っています。

**■対処**：  
[01_04_Install.ipynbの「Elasticsearchの設定」](01_04_Install.ipynb#Elasticsearchの設定)にある「カーネルパラメータを変更」のコマンドを実行し、  
カーネルパラメータvm.max_map_countを設定してください。

### Slowログが出力されている
**■項目**：[Slowログが無いか？](#Slowログが無いか？)  
**■症状**：Slowログが出力されている

**■原因/対処**：  
サーバのスペック不足、サーバ数の不足、ヒープの枯渇、インデックスの作成漏れ等が考えられます。

[診断](#診断)結果以外に、**Profile API**でクエリの実行状況を調査できます。  
Profile APIは、通常の検索query内に  
**"profile": true **  
のパラメータを追加するだけで動作します。  

例えば2015年1月分の東京のデータを検索するクエリでProfileを有効にするには次のようにします。

In [20]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.01/_search?pretty" -d @- << EOF
{
  "profile": true,
  "query": {
    "term" : { "location" : "tokyo" } 
  }
}
EOF

Profile APIの詳細は、Elasticsearch Referenceの[Profile API](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/search-profile.html)を参照してください。

ただし、Profileの結果は時間が立つと変化してしまうため注意が必要です。  
これは、データがどのshardに入るのかは、データのハッシュ値によって変化するため、  
クラスタ状況によってはデータの再配置が起きて、Shard間でデータの移動が行われる可能性があるためです。  
（例：ある特定ノードで残容量がwater markを超過したためにshardの移動が発生した。など)  


**[診断](#診断)結果とProfile結果を踏まえ、以下の節の内容を確認して対処してください。**

#### クラスタ内のサーバのCPU・メモリが枯渇している
**■項目**：[クラスタ内のサーバのCPU・メモリが枯渇していないか？](#クラスタ内のサーバのCPU・メモリが枯渇していないか？)  
**■症状**：CPUかメモリのどちらかが枯渇している

**■原因**：  
サーバで動作しているElasticsearchかそれ以外のプロセスに対し、サーバのスペックが足りていない。

**■対処**：  
- 他のプロセスによるCPUやメモリの使用率が大きい場合は、プロセスの停止か別サーバへの移動を検討してください。
- ElasticsearchのCPU使用率が大きい場合は、サーバへのメモリ追加か、サーバ台数の追加を検討してください。
  - AWSのインスタンスサイズを変更する場合は、AWS公式ページの[インスタンスのサイズ変更](http://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/ec2-instance-resize.html)を参照してください。
  - サーバ台数の追加は[01_50_Add_Data_Node.ipynb](01_50_Add_Data_Node.ipynb)を実施してください。
- Elasticsearchのメモリ使用量は、Javaの起動オプションに設定した**"-Xmx"**（最大ヒープサイズ）の値を超えることはありません。  
この使用量が想定外であった場合は次を確認してください。
  1. 実際に稼働しているプロセスのオプションを確認します。  
  診断の章の[プロセスが稼働しているか？](#プロセスが稼働しているか？)の節で実施したpsコマンドの出力に、Javaプロセスの引数が出ているはずです。"-Xmx" オプションの内容が想定通りか確認してください。
  2. 想定と異なる場合、Elasticsearch用のJava起動オプションの設定が正しいか確認してください。次のコマンドで確認できます。  
  再設定が必要な場合は[01_04_Install.ipynbの「Elasticsearchの設定」](01_04_Install.ipynb#Elasticsearchの設定)を実施してください。

In [21]:
!ansible all -m shell -a "sudo grep -e '^-Xmx' /etc/elasticsearch/jvm.options" -i ./hosts -u $USER --private-key=$KEYPATH

#### ヒープが枯渇している
**■項目**：[ヒープが枯渇していないか？](#ヒープが枯渇していないか？)  
**■症状**：ヒープの使用量が100%近くに達している、またはGC数が大きい

**■原因**：  
ヒープサイズの設定漏れ、またはクラスタの負荷状況にサーバ台数が及んでいない。

**■対処**：  
- ヒープサイズの設定が想定通りか確認してください。
  1. 実際に稼働しているプロセスのオプションを確認します。  
  診断の章の[プロセスが稼働しているか？](#プロセスが稼働しているか？)の節で実施したpsコマンドの出力に、Javaプロセスの引数が出ているはずです。"-Xmx" オプションの内容が想定通りか確認してください。
  2. 想定と異なる場合、Elasticsearch用のJava起動オプションの設定が正しいか確認してください。次のコマンドで確認できます。  
  再設定が必要な場合は[01_04_Install.ipynbの「Elasticsearchの設定」](01_04_Install.ipynb#Elasticsearchの設定)を実施してください。


In [22]:
!ansible all -m shell -a "sudo grep -e '^-Xmx' /etc/elasticsearch/jvm.options" -i ./hosts -u $USER --private-key=$KEYPATH

- ヒープサイズが想定通りで、それでもヒープの使用量が100%に近くなってしまっている場合は、Data Nodeの追加を検討してください。  
[01_50_Add_Data_Node.ipynb](01_50_Add_Data_Node.ipynb)を実施してください。

#### 実行中の重いタスクがある
**■項目**：[実行中の重いタスクが無いか？](#実行中の重いタスクが無いか？)  
**■症状**：想定外のタスクや実行時間の長いタスクがある、または大量のタスクが滞留している

**■原因**：  
他のタスクがマシンリソースを圧迫し、クエリの応答を遅くしている。

**■対処**：  
- 呼び出されているタスクの種類や量が想定外の場合、Elasticsearchを呼び出している側のアプリケーションの挙動を確認してください。
- 他のタスクがindexingであった場合、マシンへの負荷が高まり応答に時間がかかる場合があります。詳しくはElasticsearch Referenceの[Indexing Performance Tips](https://www.elastic.co/guide/en/elasticsearch/guide/2.x/indexing-performance.html)を参照してください。
- タスクがキャンセル処理をサポートしている場合、APIによるキャンセルが可能です。詳しくはElasticsearch Referenceの[Task Cancellation](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/tasks.html#_task_cancellation)を参照してください。


#### インデックスが作成されていない
**■項目**：[インデックスが作成されているか？](#インデックスが作成されているか？)  
**■症状**：インデックスが作成されていない

**■原因**：  
インデックスでデータが管理されていないため、不要な範囲の検索が走行しており、遅くなっている。

**■対処**：  
- インデックスを設定し、インデックス名が想定通りになっているか（例えば日付インデックスであればインデックス名に日付文字列が入るか）を確認してください。  
インデックスについては[05_Indexing.ipynb](05_Indexing.ipynb)を参照してください。

#### Shard数が足りていない
**■項目**：[Shard数が設定通りか？](#Shard数が設定通りか？)  
**■症状**：Shard数が設定したはずの数より不足している

**■原因**：  
テンプレートによるShard数の設定漏れ。またはData Nodeを増やしたときにShard数も増やす処理を忘れている。  
結果、存在するData Node群での検索処理が十分に並列化できず、遅くなっている。

**■対処**：  
- 作成済みのインデックスのシャード数は変更することができません。  
シャード数を増やしたインデックスを作成した後、作成したインデックスにデータを移動させてください。(reindex)  
次のコマンドを実行して新しいインデックスのシャード数を設定してください。

- reindexを実行し、新しいインデックスにデータを移動します。  
reindexについては[05_Indexing.ipynb](05_Indexing.ipynb#reindex-APIを使った投入後データの加工方法)を参考にしてください。  
このreindexの説明ではデータを加工していますが、"script"や"pipeline"を指定しないようにすることで加工せずにデータの移動のみ行うことができます。